In [ ]:
#hide

%load_ext autoreload
%autoreload 2

# Welcome to nangs

> Solving Partial Differential Equations with Neural Networks.

Nangs is a Python library built on top of Pytorch to solve Partial Differential Equations.

Our objective is to develop a new tool for simulating nature, using Neural Networks as solution approximation to Partial Differential Equations, increasing accuracy and optimziation speed while reducing computational cost.

Read our [paper](https://arxiv.org/abs/1912.04737) to know more.

## Installing

nangs is on PyPI so you can just run:

`pip install nangs`

## Getting Started

Let's assume we want to solve the following PDE:

$$\frac{\partial \phi}{\partial t} + u \frac{\partial \phi}{\partial x} = 0$$

Different numerical techniques that solve this problem exist, and all of them are based on finding an approximate function that satisfies the PDE. Traditional numerical methods discretize the domain into small elements where a form of the solutions is assumed (for example, a constant) and then the final solution is composed as a piece-wise, discontinuous function.

Nangs uses the property of neural networks (NNs) as universal function approximators to find a continuous and derivable solution to the PDE, that requires significant less computing resources compared with traditional techniques and with the advantage of including the free-parameters as part of the solution.

The independen variables (i.e, $x$ and $t$) are used as input values for the NN, and the solution (i.e. $\phi$) is the output. In order to find the solution, at each step the NN outputs are derived w.r.t the inputs. Then, a loss function that matches the PDE is built and the weights are updated accordingly. If the loss function goes to zero, we can assume that our NN is indeed the solution to our PDE.

In [ ]:
#hide

import math
import numpy as np 
import matplotlib.pyplot as plt 

import torch
cuda = False
device = torch.device("cuda:0" if torch.cuda.is_available() and cuda else "cpu")



if False:
    # import nangs
    from nangs import PDE
    from nangs.bocos import PeriodicBoco, DirichletBoco

    # define custom PDE
    class MyPDE(PDE):
        def __init__(self, inputs=None, outputs=None, params=None):
            super().__init__(inputs, outputs, params)
        def computePdeLoss(self, grads, inputs, params): 
            # here is where the magic happens
            dpdt, dpdx = grads['p']['t'], grads['p']['x']
            u = params['u']
            return dpdt + u*dpdx

    # instanciate pde
    pde = MyPDE(inputs=['x', 't'], outputs=['p'], params=['u'])

    # define input values for training
    x = np.linspace(0,1,30)
    t = np.linspace(0,1,20)
    u = np.array([1.0])
    pde.setTrainValues({'x': x, 't': t, 'u': u})

    # define input values for validation
    x_v = np.linspace(0,1,50)
    t_v = np.linspace(0,1,30)
    pde.setValValues({'x': x_v, 't': t_v})

    # periodic b.c for the space dimension
    x1, x2 = np.array([0]), np.array([1])
    boco = PeriodicBoco(pde.inputs, {'x': x1, 't': t}, {'x': x2, 't': t})
    pde.addBoco(boco)

    # initial condition (dirichlet for temporal dimension)
    p0 = np.sin(2.*math.pi*x)
    boco = DirichletBoco(pde.inputs, pde.outputs, {'x': x, 't': np.array([0])}, {'p': p0})
    pde.addBoco(boco)

    # define solution topology
    mlp = {'layers': 3, 'neurons': 100, 'activations': 'relu'}
    pde.buildModel(mlp)

    # set optimization parameters
    pde.setSolverParams(lr=0.01, epochs=20, batch_size=50)

    # find the solution
    path = 'best_model.pth'
    pde.solve(device, path) 

    # evaluate the solution
    pde.model.load_state_dict(torch.load(path))
    x = np.linspace(0,1,50)
    t = np.linspace(0,1,4)
    for _t in t:
        p0 = np.sin(2.*math.pi*(x-u*_t))
        p = pde.eval({'x': x, 't': np.array([_t])}, device)
        plt.plot(x, p, 'k')
        plt.plot(x, p0)
        plt.show()

## Step by step guide

Let's go through the code step by step. First, we import the nangs module to acces its predefined classes and operations to solve PDEs with NNs. The base class we work with is the PDE class, where we have all the methods to set the data and find a solution. Then, the bocos module gives us access to the different boundary conditions implemented.

## Copyright

Copyright 2020 onwards, SensioAI. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this project's files except in compliance with the License. A copy of the License is provided in the LICENSE file in this repository.